In [9]:
import asyncio , re , httpx
from bs4 import BeautifulSoup
from typing import Optional , AsyncIterator , List
import urllib.parse
import requests
from dotenv import load_dotenv
import os, sys
from tenacity import retry, stop_after_attempt,wait_exponential
import certifi
from playwright.async_api import async_playwright
import asyncio
import os
from typing import Optional
from urllib.parse import quote , urlparse , urljoin
import httpx
import random

In [2]:
load_dotenv()

True

In [21]:
print(certifi.where())


/home/longlab/Research_Projects/rheumatology_llm/rheum_project/robust_py311/lib/python3.11/site-packages/certifi/cacert.pem


In [3]:

cwd = os.getcwd()
parent_folder = os.path.abspath(os.path.join(cwd, ".."))
if parent_folder not in sys.path:
    sys.path.append(parent_folder)
from functions.paper_to_doi import get_info_from_doi


2025-06-22 17:31:16.229827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750635076.291227    5590 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750635076.307815    5590 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750635076.432555    5590 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750635076.432584    5590 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750635076.432586    5590 computation_placer.cc:177] computation placer alr

In [ ]:
# class Easy_Publisher_PDF_Resolver:
#     """
#     Resolve a DOI to a direct PDF URL for Springer, OUP, Hindawi, F1000, etc.
#     Raises NeedsBrowser when the easy path fails.
#     """
    
#     _SPRINGER_HOST     = "link.springer.com"
#     _OUP_HOST_RE       = re.compile(r"^https?://academic\.oup\.com/")
#     _F1000_HOST_RE     = re.compile(r"^https?://(?:f1000research|wellcomeopenresearch|gatesopenresearch)\.org/")
#     _HINDAWI_DOWNLOAD  = re.compile(r"^https?://downloads\.hindawi\.com/")
#     _HINDAWI_LANDING   = re.compile(r"^https?://(?:www\.)?hindawi\.com/")

    
#     _ANCHOR_PDF_CLASS_RE = re.compile(r"pdf|download", re.I)  # class / id hints


#     class NeedsBrowser(RuntimeError):
#         """Raised when the resolver needs a browser to resolve the URL."""
#         pass
    
    
#     def __init__(self,*,
#                  headers: Optional[dict] = None,
#                  timeout: int = 30):
#         self.headers = headers or {
#             "User-Agent": "Mozilla/5.0 (easy-pdf-resolver/0.2)"
#         }
#         self.timeout = timeout
#         self._client : Optional[httpx.AsyncClient] = None
      
#     async def __aenter__(self): 
#         self._client = httpx.AsyncClient(
#             headers=self.headers,
#             follow_redirects=True, 
#             timeout=self.timeout
#         )
        
#         return self
    
#     async def __aexit__(self, exc_type, exc, traceback):
#         await self._client.aclose()
#         self._client = None
        
#     def _require_client(self) -> httpx.AsyncClient:
#         if self._client is None:
#             # allow use without 'async with' by lazily creating a client
#             self._client = httpx.AsyncClient(
#                 headers=self.headers,
#                 follow_redirects=True,
#                 timeout=self.timeout
#             )
#         return self._client

    
#     async def _is_pdf(self, url: str) -> bool:
#         client = self._require_client()
#         r = await client.head(url)
#         return (
#             r.status_code == 200 and
#             r.headers.get("content-type", "").startswith("application/pdf")
#         )
    
#     @staticmethod
#     def _extract_meta_pdf(html: str) -> Optional[str]:
#         """
#         Extract the PDF URL advertised in a page’s ``<meta>`` tag.

#         Many scholarly-publisher landing pages include
#         ``<meta name="citation_pdf_url" content="https://…/article.pdf">``  
#         inside their ``<head>`` section.  This helper pulls that value out
#         so the scraper can download the PDF without further heuristics.

#         Args:
#         html (str):
#             Raw HTML source of the article landing page
#             (typically obtained via ``await http_client.get(url).text``).

#         Returns:
#             Optional[str]:
#                 • The PDF URL found in the ``content`` attribute of the first
#                 ``<meta name="citation_pdf_url">`` tag, **or**  
#                 • ``None`` if no such tag exists (caller should try another
#                 strategy, e.g. Playwright).

#         Example:
#             >>> landing_html = client.get("https://academic.oup.com/jn/article/...", timeout=30).text
#             >>> pdf_url = _extract_meta_pdf(landing_html)
#             >>> print(pdf_url)
#             https://academic.oup.com/jn/article-pdf/123/4/1234/9876543/jn.123.4.1234.pdf

#         Notes:
#             * Uses BeautifulSoup’s default parser (``html.parser``) to avoid
#             external dependencies.  Switch to ``"lxml"`` if you already have
#             lxml installed and want a performance boost.
#             * Only the **first** matching tag is returned; this is sufficient
#             for OUP, Wiley HTML, and most CrossRef-compliant publishers.

#         """
#         soup = BeautifulSoup(html, "html.parser")
#         tag  = soup.find("meta", attrs={"name": "citation_pdf_url"})
#         return tag["content"] if tag else None
    
    
#     @classmethod
#     def _extract_anchor_pdf(cls, html: str, base_url: str) -> Optional[str]:
#         """Return absolute URL from the *best* `<a>` that seems to be a PDF link.

#         Heuristics (checked in order):
#         1. `href` ends with `.pdf` (common for Springer / OUP).
#         2. `href` contains `/pdf` *and* anchor text or class‑attribute mentions
#            "pdf" or "download" (F1000 style: `/pdf?article_uuid=…`).
#         3. Anchor `class` / `id` matches /pdf|download/ even if `href` is just
#            a relative `/content/pdf/…` path.
#         """
#         soup = BeautifulSoup(html, "html.parser")

#         def make_abs(href: str) -> str:
#             return urllib.parse.urljoin(base_url, href)

#         # Pass 0 – explicit OUP pattern /article-pdf/ ---------------------------
#         for a in soup.find_all("a", href=True):
#             href = a["href"].strip()
#             if "/article-pdf/" in href.lower():
#                 return make_abs(href)

#         # Pass 1 – href ends with .pdf -----------------------------------------
#         for a in soup.find_all("a", href=True):
#             href = a["href"].strip()
#             if href.lower().endswith(".pdf"):
#                 return make_abs(href)

#         # Pass 2 – href contains "pdf" + textual or class cues -----------------
#         for a in soup.find_all("a", href=True):
#             href = a["href"].strip()
#             href_lower = href.lower()
#             if "pdf" in href_lower:
#                 text = a.get_text(" ", strip=True).lower()
#                 classes = " ".join(a.get("class", [])).lower()
#                 ident   = a.get("id", "").lower()
#                 if ("pdf" in text or "download" in text or cls._ANCHOR_HINT_RE.search(classes) or cls._ANCHOR_HINT_RE.search(ident)):
#                     return make_abs(href)

#         # Pass 3 – only class/id hints -----------------------------------------
#         for a in soup.find_all("a", href=True):
#             classes = " ".join(a.get("class", []))
#             ident   = a.get("id", "")
#             if cls._ANCHOR_HINT_RE.search(classes) or cls._ANCHOR_HINT_RE.search(ident):
#                 return make_abs(a["href"].strip())

#         return None

        
       
        
#     async def get_pdf(self, doi: str) -> str:
        
#         """
#         Return a direct PDF URL for the given DOI.
#         Raises EasyPDFResolver.NeedsBrowser if the easy strategies fail.
#         """
#         client = self._client or httpx.AsyncClient(headers=self.headers, follow_redirects=True, timeout=self.timeout)
#         landing = str((await client.get(f"https://doi.org/{doi}")).url)
        
#         if self._SPRINGER_HOST in landing:
#             candidates = [
#                 f"https://{self._SPRINGER_HOST}/content/pdf/{doi}.pdf",
#             ]
#             # /fulltext.html → .pdf
#             if landing.endswith("fulltext.html") or landing.endswith("fulltext.htm"):
#                 candidates.append(landing.rsplit("/", 1)[0] + ".pdf")
#             # article path without extension → add .pdf
#             elif landing.endswith(".html"):
#                 candidates.append(landing[:-5] + ".pdf")
#             else:
#                 if "/article/" in landing and not landing.endswith(".pdf"):
#                     candidates.append(landing + ".pdf")

#             for pdf_url in candidates:
#                 if await self._is_pdf(pdf_url):
#                     return pdf_url

#             # 🚀 NEW: scrape the Download‑PDF button

#         # ------------------------------------------------------------------
#         # Hindawi – DOI redirect or downloads.hindawi.com
#         # ------------------------------------------------------------------
#         if self._HINDAWI_DOWNLOAD.match(landing) or self._HINDAWI_LANDING.match(landing):
#             for cand in {landing, f"https://doi.org/{doi}"}:
#                 if await self._is_pdf(cand):
#                     return cand
#             # try landing→html parse as last resort
#             html = (await client.get(landing)).text
#             meta = self._extract_meta_pdf(html)
#             if meta and await self._is_pdf(meta):
#                 return meta

#         # ------------------------------------------------------------------
#         # F1000Research family – JSON API
#         # ------------------------------------------------------------------
#         if self._F1000_HOST_RE.match(landing):
#             article_id = doi.split("/")[-1]
#             host = urllib.parse.urlparse(landing).hostname
#             api_url = f"https://api.{host}/article/{article_id}"
#             try:
#                 pdf_url = (await client.get(api_url)).json()["data"]["pdf_url"]
#                 if await self._is_pdf(pdf_url):
#                     return pdf_url
#             except Exception:
#                 pass

#         # ------------------------------------------------------------------
#         # OUP – meta tag only
#         # ------------------------------------------------------------------
#         if self._OUP_HOST_RE.match(landing):
#             html = (await client.get(landing)).text
#             meta_link = self._extract_meta_pdf(html)
#             if meta_link and await self._is_pdf(meta_link):
#                 return meta_link
#         try:
#             html = (await client.get(landing)).text
#             anchor_link = self._extract_anchor_pdf(html, landing)
#             if anchor_link and await self._is_pdf(anchor_link):
#                 return anchor_link
#         except httpx.HTTPError:
#             pass 
#         # ------------------------------------------------------------------
#         # Escalate
#         # ------------------------------------------------------------------
#         raise self.NeedsBrowser(f"Browser required for {doi} → {landing}")


In [ ]:
class PDFResolver:
    _SPRINGER_HOST = "link.springer.com"
    _OUP_HOST_RE = re.compile(r"^https?://academic\.oup\.com/")
    _F1000_HOST_RE = re.compile(r"^https?://(?:f1000research|wellcomeopenresearch|gatesopenresearch)\.org/")
    _HINDAWI_DOWNLOAD_RE = re.compile(r"^https?://downloads\.hindawi\.com/")
    _HINDAWI_LANDING_RE = re.compile(r"^https?://(?:www\.)?hindawi\.com/")
    _ANCHOR_HINT_RE = re.compile(r"""(?x)                         # verbose mode for clarity
    (
        pdf
        | full[\s_-]*text
        | full[\s_-]*article
        | view[\s_-]*article
        | view[\s_-]*pdf
        | read[\s_-]*(this[\s_-]*)?article
        | article[\s_-]*as[\s_-]*pdf
        | open[\s_-]*access
        | download
        | dl[\s_-]*(pdf|article)?
        | link[\s_-]*to[\s_-]*pdf
        | show[\s_-]*pdf
        | get[\s_-]*pdf
        | pdf[\s_-]*download
        | primary[\s_-]*document
        | main[\s_-]*article
        | publication[\s_-]*file
        | document[\s_-]*view
        | article[\s_-]*file
        | content[\s_-]*pdf
        | view[\s_-]*full[\s_-]*text
        | citation[\s_-]*pdf[\s_-]*url
    )
    """,
    re.I
)
    wiley_token = os.getenv("wiley_api_token")

    class CantDownload(RuntimeError):
        pass

    def __init__(self, *, headers: Optional[dict] = None, timeout: int = 30):
        self.headers = headers or {"User-Agent": "Mozilla/5.0 (easy-pdf-resolver/1.2)"}
        self.timeout = timeout
        self._client: Optional[httpx.AsyncClient] = None

    async def __aenter__(self):
        self._client = httpx.AsyncClient(headers=self.headers, follow_redirects=True, timeout=self.timeout)
        return self

    async def __aexit__(self, exc_type, exc, tb):
        if self._client:
            await self._client.aclose()
            self._client = None

    def _client_required(self) -> httpx.AsyncClient:
        if self._client is None:
            self._client = httpx.AsyncClient(headers=self.headers, follow_redirects=True, timeout=self.timeout)
        return self._client

    async def get_pdf(self, doi: str) -> str:
        client = self._client_required()
        landing = str((await client.get(f"https://doi.org/{doi}")).url)

        if self._SPRINGER_HOST in landing:
            for url in self._springer_candidates(landing, doi):
                if await self._is_pdf(url):
                    return url

        if self._HINDAWI_DOWNLOAD_RE.match(landing) or self._HINDAWI_LANDING_RE.match(landing):
            for url in (landing, f"https://doi.org/{doi}"):
                if await self._is_pdf(url):
                    return url

        if self._F1000_HOST_RE.match(landing):
            api_pdf = await self._f1000_pdf(landing, doi)
            if api_pdf and await self._is_pdf(api_pdf):
                return api_pdf

        if self._OUP_HOST_RE.match(landing):
            oup_pdf = await self._oup_pdf(landing)
            if oup_pdf:
                return oup_pdf
            
        if "onlinelibrary.wiley.com" in landing:
            wiley_pdf = await self._wiley_pdf(landing, doi)
            if wiley_pdf:
                return wiley_pdf
        
        html = (await client.get(landing)).text
        anchor = self._extract_anchor_pdf_score(html, landing)
        if anchor:
            return anchor

        cr_pdf = self._crossref_fallback(doi)
        if cr_pdf:
            return cr_pdf
        
        browser_pdf = await self.fetch_pdf_with_browser(doi)
        if browser_pdf:
            return browser_pdf
        

        raise self.CantDownload(f"Cant Downlaod  {doi} → {landing}")

    async def _springer_candidates(self, landing: str, doi: str) -> List[str]:
        base = f"https://{self._SPRINGER_HOST}"
        candidates = [f"{base}/content/pdf/{doi}.pdf"]
        if landing.endswith(("fulltext.html", "fulltext.htm")):
            candidates.append(landing.rsplit("/", 1)[0] + ".pdf")
        elif landing.endswith(".html"):
            candidates.append(landing[:-5] + ".pdf")
        elif "/article/" in landing and not landing.endswith(".pdf"):
            candidates.append(landing + ".pdf")
        return candidates

    async def _f1000_pdf(self, landing: str, doi: str) -> Optional[str]:
        client = self._client_required()
        article_id = doi.split("/")[-1]
        host = urllib.parse.urlparse(landing).hostname
        api_url = f"https://api.{host}/article/{article_id}"
        try:
            r = await client.get(api_url)
            r.raise_for_status()
            return r.json()["data"]["pdf_url"]
        except Exception:
            return None

    async def _oup_pdf(self, landing: str) -> Optional[str]:
        client = self._client_required()
        html = (await client.get(landing)).text
        meta = self._extract_meta_pdf(html)
        if meta:
            return meta
        anchor = self._extract_anchor_pdf(html, landing)
        return anchor

    async def _wiley_pdf(self , landing:str , doi:str) -> Optional[str]:
        client = self._client_required()
        
        #1 TDM Api
        api_url = f"https://api.wiley.com/tdm/v1/articles?{doi}/pdf"
        r = await client.get(
            api_url,
            headers= {"Authorization": f"Bearer {self.wiley_token}"})
        print(f"[Wiley] API status: {r.status_code}, content-type: {r.headers.get('content-type')}")
        print(f"[Wiley] Token present? {'yes' if self.wiley_token else 'no'}")
        
        if r.status_code == 200 and r.headers.get("content-type", "").startswith("application/pdf"):
            return api_url
        
        #pdfdirect 
        
        pdf_url = f"https://onlinelibrary.wiley.com/doi/pdfdirect/{doi}"
        r2 = await client.get(pdf_url)
        if r2.status_code == 200 and r2.headers.get("content-type", "").startswith("application/pdf"):
            return pdf_url
        
        #htmlparse 
        
        html = (await client.get(landing)).text
        soup = BeautifulSoup(html, "html.parser")
        btn = soup.find("a", class_="pdf-download-link", href=True)
        if btn:
            return urllib.parse.urljoin(landing, btn["href"])
        return None 
    
    async def _tand_pdf(self , landing:str , doi:str) -> Optional[str]:
        client = self._client_required()
        
        #direct pdf endpoint 
        
        pdf_endpoint = f"https://www.tandfonline.com/doi/pdf/{doi}"
        resp = await client.get(pdf_endpoint)
        if resp.status_code == 200 and resp.headers.get("content-type", "").startswith("application/pdf"):
            return pdf_endpoint
        
        #meta tag
        html = (await client.get(landing)).text
        if (meta:= self._extract_meta_pdf(html)):
            return urllib.parse.urljoin(landing, meta)
        
        if (anchor := self._extract_anchor_pdf(html, landing)):
            return anchor
        return None
        
    async def _sage_pdf(self, landing: str, doi: str) -> Optional[str]:
        client = self._client_required()
        
        #pdf endpoint
        
        pdf_url = f"https://journals.sagepub.com/doi/pdf/{doi}"
        r = await client.get(pdf_url)
        if r.status_code == 200 and r.headers.get("content-type", "").startswith("application/pdf"):
            return pdf_url
        
        html = (await client.get(landing)).text
        if (meta:= self._extract_meta_pdf(html)):
            return urllib.parse.urljoin(landing, meta)
        
        if (anchor := self._extract_anchor_pdf(html, landing)):
            return anchor
        return None
        
        
    async def _karger_pdf(self, landing: str, doi: str) -> Optional[str]:
        clinet = self._client_required()
        
        #must get article id 
        
        html = (await clinet.get(landing)).text
        soup = BeautifulSoup(html, "html.parser")
        tag = soup.find("meta", attrs={"name": "dc.identifier"})
        if not tag or not tag.get("data-article-id"):
            return None
        article_id = tag["data-article-id"]
        return f"https://www.karger.com/Article/Pdf/{article_id}"
       
    async def _is_pdf(self, url: str) -> bool:
        client = self._client_required()
        try:
            r = await client.head(url)
            if r.status_code in (403, 405):
                r = await client.get(url, headers={"Range": "bytes=0-0"})
            return r.status_code == 200 and r.headers.get("content-type", "").startswith("application/pdf")
        except Exception:
            return False

    @staticmethod
    def _extract_meta_pdf(html: str) -> Optional[str]:
        tag = BeautifulSoup(html, "html.parser").find("meta", attrs={"name": "citation_pdf_url"})
        return tag["content"].strip() if tag and tag.get("content") else None

    def _extract_anchor_pdf(self, html: str, base_url: str) -> Optional[str]:
        soup = BeautifulSoup(html, "html.parser")
        join = lambda h: urllib.parse.urljoin(base_url, h)
        for a in soup.find_all("a", href=True):
            href = a["href"].strip()
            lower = href.lower()
            full_url = join(href)
            if lower.endswith(".pdf") or "/article-pdf/" in lower or "/advance-article-pdf/" in lower:
                return full_url
            text = a.get_text(" ").lower()
            cls = " ".join(a.get("class", [])).lower()
            ident = a.get("id", "").lower()
            if "download pdf" in text or self._ANCHOR_HINT_RE.search(cls) or self._ANCHOR_HINT_RE.search(ident):
                return full_url
        return None
    
    def _extract_anchor_pdf_score(self, html: str, base_url: str) -> Optional[str]:
        soup = BeautifulSoup(html, "html.parser")
        join = lambda h: urllib.parse.urljoin(base_url, h)
        best = None

        for a in soup.find_all("a", href=True):
            href = a["href"].strip()
            lower = href.lower()
            full_url = join(href)

            text = a.get_text(" ").lower()
            cls = " ".join(a.get("class", [])).lower()
            ident = a.get("id", "").lower()
            title = a.get("title", "").lower()
            aria = a.get("aria-label", "").lower()

            score = 0
            if lower.endswith(".pdf"):
                score += 3
            if "/article-pdf/" in lower or "/advance-article-pdf/" in lower:
                score += 2
            if "pdf" in text:
                score += 2
            if re.search(self._ANCHOR_HINT_RE, cls) or re.search(self._ANCHOR_HINT_RE, ident):
                score += 1
            if "full text" in text or "read article" in text:
                score += 1
            if "pdf" in title or "pdf" in aria:
                score += 1

            if score >= 3:  # only consider fairly confident ones
                return full_url
            elif not best and score > 0:
                best = full_url

        return best
    @staticmethod
    def _crossref_fallback(doi: str) -> Optional[str]:
        try:
            r = requests.get(f"https://api.crossref.org/works/{doi}", timeout=15)
            r.raise_for_status()
            data = r.json()
        except Exception:
            return None
        links = data.get("message", {}).get("link", [])
        best = None
        for link in links:
            ct = link.get("content-type")
            url = link.get("URL")
            if ct == "text/html":
                return url
            if ct == "application/pdf" and best is None:
                best = url
            if url and url.endswith(".pdf") and best is None:
                best = url
        return best
    
    async def fetch_pdf_with_browser(self ,doi: str) -> Optional[str]:
        EZPROXY_PREFIX = "https://ezproxy.lib.ucalgary.ca/login?url="
        JOURNAL_PDF_SELECTORS = {
        "bmj.com": 'a[title="Download this article as a PDF"]',
        "thelancet.com": 'a.download-pdf-link',
        "nejm.org": 'a[href*="pdf"]',
        "sciencedirect.com": 'a.pdf-download',
        "jamanetwork.com": 'a[href*="/pdf"]',
        "nature.com": 'a[data-track-action="download pdf"]',
        "springer.com": 'a[href$=".pdf"]',
        "oup.com": 'a[href$=".pdf"]',
        "frontiersin.org": 'a.article-pdf-download',
        "tandfonline.com": 'a[href*="/pdf"]',
        "sagepub.com": 'a.article-tools-download',
        "karger.com": 'a[href$=".pdf"]',
        # Add more as needed...
    }
        USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) Gecko/20100101 Firefox/125.0",
]
        client = self._client_required()
        landing_url = str((await client.get(f"https://doi.org/{doi}")).url)
        proxied_url = EZPROXY_PREFIX + quote(landing_url, safe="")
        domain = urlparse(landing_url).netloc

        username = os.getenv("uni_username")
        password = os.getenv("uni_password")
        if not username or not password:
            raise RuntimeError("Missing EZProxy credentials.")

        async with async_playwright() as p:
            browser = await p.chromium.launch(
                headless=True, 
                args=[ "--disable-blink-features=AutomationControlled",
                    "--disable-web-security",
                    "--no-sandbox",
                    "--disable-gpu"
                ])
            context = await browser.new_context(user_agent=random.choice(USER_AGENTS))
            page = await context.new_page()

            # EZProxy login page
            if "login" in page.url.lower():
                await page.goto(proxied_url)
                await page.fill('input[name="user"]', username)
                await page.fill('input[name="pass"]', password)
                await page.click('input[type="submit"]')
                await page.wait_for_load_state("networkidle")

            # Journal-specific selector
            selector = JOURNAL_PDF_SELECTORS.get(domain)
            if selector:
                try:
                    await page.wait_for_selector(selector, timeout=5000)
                    pdf_link = await page.locator(selector).get_attribute("href")
                    if pdf_link:
                        return urljoin(page.url, pdf_link)
                except Exception:
                    pass  # fallback to anchor hint matching

            # Fallback using _ANCHOR_HINT_RE
            html = await page.content()
            soup = BeautifulSoup(html, "html.parser")
            for a in soup.find_all("a", href=True):
                href = a["href"].strip()
                text = a.get_text(" ").lower()
                cls = " ".join(a.get("class", [])).lower()
                id_ = a.get("id", "").lower()
                title = a.get("title", "").lower()
                aria = a.get("aria-label", "").lower()

                if (
                    href.lower().endswith(".pdf") or
                    _ANCHOR_HINT_RE.search(text) or
                    _ANCHOR_HINT_RE.search(href) or
                    _ANCHOR_HINT_RE.search(cls) or
                    _ANCHOR_HINT_RE.search(id_) or
                    _ANCHOR_HINT_RE.search(title) or
                    _ANCHOR_HINT_RE.search(aria)
                ):
                    return urljoin(page.url, href)

            return None
        

In [51]:
def resolve_pmcid_from_doi(doi: str, email: str = "you@example.com") -> Optional[str]:
    """Convert DOI to PMCID using NCBI ID Converter API."""
    url = f"https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?tool=biorxiv-utils&email={email}&ids={doi}"
    try:
        resp = requests.get(url, timeout=15)
        resp.raise_for_status()
        match = re.search(r'pmcid="(PMC[0-9]+)"', resp.text)
        return match.group(1) if match else None
    except Exception as e:
        print(f"❌ DOI to PMCID resolution failed: {e}")
        return None

def fetch_pmc_pdf(pmcid: str) -> Optional[bytes]:
    # url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmcid}/pdf"
    # url = f"https://pmc.ncbi.nlm.nih.gov/articles/{pmcid}/pdf"
    # url = f"https://www-ncbi-nlm-nih-gov.ezproxy.lib.ucalgary.ca/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{pmcid}/unicode"
    url = f"https://www-ncbi-nlm-nih-gov.ezproxy.lib.ucalgary.ca/pmc/articles/{pmcid}/pdf"

    

    try:
        with httpx.Client( verify= False, timeout=60,follow_redirects=True) as client:
            response = client.get(url.strip())
            response.raise_for_status()
            return response.content
    except Exception as e:
        print(f"❌ Failed to fetch PMC PDF: {e}")
        return None


In [13]:
import httpx
import xml.etree.ElementTree as ET
from typing import Optional

def extract_isrctn_from_doi(doi: str) -> Optional[str]:
    """
    Extract the ISRCTN number from a DOI like "10.1186/isrctn04296019".
    """
    if "isrctn" in doi.lower():
        return doi.lower().split("isrctn")[-1]
    return None

def fetch_isrctn_trial_data(doi: str, format_: str = "who") -> Optional[dict]:
    """
    Fetch ISRCTN trial data given a DOI containing an ISRCTN number.

    Args:
        doi: The DOI containing the ISRCTN, e.g. "10.1186/isrctn04296019"
        format_: One of "default", "ukctg", or "who"

    Returns:
        A dictionary of parsed trial data, or None if not found or failed.
    """
    isrctn = extract_isrctn_from_doi(doi)
    if not isrctn:
        print("❌ ISRCTN not found in DOI")
        return None

    url = f"https://www.isrctn.com/api/trial/{isrctn}/format/{format_}"
    try:
        r = httpx.get(url, timeout=15)
        r.raise_for_status()
    except httpx.RequestError as e:
        print(f"❌ HTTP error: {e}")
        return None

    try:
        root = ET.fromstring(r.text)
        return {elem.tag: elem.text for elem in root.iter() if elem.text}
    except ET.ParseError:
        print("❌ Failed to parse XML")
        return None


In [19]:
test_dois = [
    "10.3390/jcm8081218" # Hindawi OA
]

In [15]:
trial_info = fetch_isrctn_trial_data(test_dois[0])

In [20]:
from pprint import pprint
async with EasyPDFResolver() as r:
    pdf_url = await r.get_pdf(test_dois[0])
    pprint(pdf_url)

CantDownload: Cant Downlaod  10.3390/jcm8081218 → https://www.mdpi.com/2077-0383/8/8/1218

In [6]:
paper_info = get_info_from_doi("10.1093/rheumatology/keab153")

In [7]:
paper_info

{'doi': '10.1093/rheumatology/keab153',
 'title': 'Rituximab plus leflunomide in rheumatoid arthritis: a randomized, placebo-controlled, investigator-initiated clinical trial (AMARA study)',
 'document_link': 'http://academic.oup.com/rheumatology/advance-article-pdf/doi/10.1093/rheumatology/keab153/36643438/keab153.pdf'}

In [ ]:
from playwright.async_api import async_playwright
import asyncio
import os
from typing import Optional
from urllib.parse import quote , urlparse , urljoin
import httpx 



async def fetch_pdf_with_browser(doi: str) -> Optional[str]:
    EZPROXY_PREFIX = "https://ezproxy.lib.ucalgary.ca/login?url="
    JOURNAL_PDF_SELECTORS = {
    "bmj.com": 'a[title="Download this article as a PDF"]',
    "thelancet.com": 'a.download-pdf-link',
    "nejm.org": 'a[href*="pdf"]',
    "sciencedirect.com": 'a.pdf-download',
    "jamanetwork.com": 'a[href*="/pdf"]',
    "nature.com": 'a[data-track-action="download pdf"]',
    "springer.com": 'a[href$=".pdf"]',
    "oup.com": 'a[href$=".pdf"]',
    "frontiersin.org": 'a.article-pdf-download',
    "tandfonline.com": 'a[href*="/pdf"]',
    "sagepub.com": 'a.article-tools-download',
    "karger.com": 'a[href$=".pdf"]',
    # Add more as needed...
}

    client = self._client_required()
    landing_url = str((await client.get(f"https://doi.org/{doi}")).url)
    proxied_url = EZPROXY_PREFIX + quote(landing_url, safe="")
    domain = urlparse(landing_url).netloc

    username = os.getenv("uni_username")
    password = os.getenv("uni_password")
    if not username or not password:
        raise RuntimeError("Missing EZProxy credentials.")

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # EZProxy login page
        if "login" in page.url.lower():
            await page.goto(proxied_url)
            await page.fill('input[name="user"]', username)
            await page.fill('input[name="pass"]', password)
            await page.click('input[type="submit"]')
            await page.wait_for_load_state("networkidle")

        # Journal-specific selector
        selector = JOURNAL_PDF_SELECTORS.get(domain)
        if selector:
            try:
                await page.wait_for_selector(selector, timeout=5000)
                pdf_link = await page.locator(selector).get_attribute("href")
                if pdf_link:
                    return urljoin(page.url, pdf_link)
            except Exception:
                pass  # fallback to anchor hint matching

        # Fallback using _ANCHOR_HINT_RE
        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")
        for a in soup.find_all("a", href=True):
            href = a["href"].strip()
            text = a.get_text(" ").lower()
            cls = " ".join(a.get("class", [])).lower()
            id_ = a.get("id", "").lower()
            title = a.get("title", "").lower()
            aria = a.get("aria-label", "").lower()

            if (
                href.lower().endswith(".pdf") or
                ANCHOR_HINT_RE.search(text) or
                ANCHOR_HINT_RE.search(href) or
                ANCHOR_HINT_RE.search(cls) or
                ANCHOR_HINT_RE.search(id_) or
                ANCHOR_HINT_RE.search(title) or
                ANCHOR_HINT_RE.search(aria)
            ):
                return urljoin(page.url, href)

        return None
    

In [ ]:
doi = "10.1038/srep40473"
landing = f"https://doi.org/{doi}"
# pdf_link = await fetch_pdf_with_browser(landing)
# print("PDF Link:", pdf_link)

TimeoutError: Page.fill: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("input[name=\"user\"]")


In [ ]:
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=2, max=20))
async def get_biorxiv_pdf_link(article_url: str) -> Optional[str]:
    pdf_url = article_url if article_url.endswith(".full.pdf") else f"{article_url}.full.pdf"
    last_exc: Optional[Exception] = None

    for hdr in _HEADERS_SETS:
        try:
            async with httpx.AsyncClient(headers=hdr, timeout=60, follow_redirects=True) as client:
                resp = await client.get(pdf_url.strip())
                resp.raise_for_status()

                if resp.headers.get("Content-Type", "").lower().startswith("application/pdf"):
                    return extract_pdf(resp.content)  # type: ignore[name-defined]

                print("⚠️  Not a PDF response (content‑type:", resp.headers.get("Content-Type"), ")")
                return None

        except httpx.HTTPStatusError as exc:
            last_exc = exc
            if exc.response.status_code in {403, 429}:
                continue
            raise
        except Exception as exc:
            last_exc = exc
            raise

    if last_exc:
        raise last_exc

    return None

async def search_pubmed_by_abstract(abstract_text: str) -> Optional[str]:
    search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": abstract_text,
        "retmode": "json",
        "retmax": 1
    }
    async with httpx.AsyncClient(timeout=30) as client:
        response = await client.get(search_url, params=params)
        if response.status_code == 200:
            data = response.json()
            id_list = data.get("esearchresult", {}).get("idlist", [])
            if id_list:
                return id_list[0]  # return first PubMed ID
    return None


In [3]:
paper_journal_info = "../research_paper_database/paper_journal_info.csv"

In [ ]:
import csv
with open(paper_journal_info, "r") as f:
    lines = csv.reader(f)
    next(lines)  # skip header
    for row in lines:
        paper_id = row[0]
        paper_title = row[1]
        paper_doi = row[2]
        paper_link = row[3]
        print(row)

['9263431', 'Tak PP; Balanescu A; Tseluyko V; Bojin S; Drescher E; Dairaghi D; Miao S; Marchesin V; Jaen J; Schall TJ; Bekker P. Chemokine receptor CCR1 antagonist CCX354-C treatment for rheumatoid arthritis: CARAT-2, a randomised, placebo controlled clinical trial. Annals of the rheumatic diseases. (2013) 72 3 pp337-344', '10.1136/annrheumdis-2011-201605', '']
['9262389', 'Fettner S; Mela C; Wildenhahn F; Tavanti M; Wells C; Mallalieu NL. Evaluation of a tocilizumab autoinjector: results of healthy volunteer bioequivalence and rheumatoid patient human factors studies. Clinical pharmacology and therapeutics. Conference: 119th annual meeting of the american society for clinical pharmacology and therapeutics, ASCPT 2018. United states. (2018) 103 Supplement 1 ppS78-S79', '10.1080/17425247.2019.1604678', 'https://tandfonline.com/doi/pdf/10.1080/17425247.2019.1604678']
['11083831', 'Tanaka Yoshiya; Millson David; Iwata Shigeru; Nakayamada Shingo. Safety and efficacy of fostamatinib in rheu